# Review: Querying an API endpoint

### Mapbox Geocoding API

Services like Google Maps and Mapbox have various APIs that let you access its services through code instead of through GUI apps. This one from Mapbox lets you look up the latitude-longitude coordinates of street addresses.

It works similarly to the earthquakes example, but with query parameters added to the URL endpoint!

**API documentation:**  
https://www.mapbox.com/api-documentation/#geocoding

**API endpoint:**  
https://api.mapbox.com/geocoding/v5/mapbox.places

**API endpoint with query parameters:**  
https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster+Hall.json?access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w

You can get your own access key by signing up for a Mapbox account, if you'd like. Here is a link for that (but don't do it now): https://www.mapbox.com/signin/?route-to=%22/account/access-tokens%22

In [1]:
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [15]:
# we have to encode the search query so that it can be passed as a URL, 
# with spaces and other special characters removed

endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

address = 'Wurster Hall'

params = {'limit': 1,
          'access_token': 'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'}

url = requests.Request('GET', endpoint+address+'.json', params=params).prepare().url
print(url)

https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster%20Hall.json?limit=1&access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w


In [16]:
# download and parse the results

response = requests.get(url)
results = response.text
data = json.loads(results)

print(data)

{'type': 'FeatureCollection', 'query': ['wurster', 'hall'], 'features': [{'id': 'poi.1359351', 'type': 'Feature', 'place_type': ['poi'], 'relevance': 1, 'properties': {'tel': '(510) 642-0831', 'address': '230 Wurster Hall #1820', 'category': 'college, university', 'landmark': True, 'maki': 'college'}, 'text': 'Wurster Hall / College of Environmental Design', 'place_name': 'Wurster Hall / College of Environmental Design, 230 Wurster Hall #1820, Berkeley, California 94720, United States', 'center': [-122.25488, 37.87082], 'geometry': {'type': 'Point', 'coordinates': [-122.25488, 37.87082]}, 'context': [{'id': 'postcode.1038777018848930', 'text': '94720'}, {'id': 'place.4062647275990170', 'wikidata': 'Q484678', 'text': 'Berkeley'}, {'id': 'region.3591', 'short_code': 'US-CA', 'wikidata': 'Q99', 'text': 'California'}, {'id': 'country.3145', 'short_code': 'us', 'wikidata': 'Q30', 'text': 'United States'}]}], 'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. Use o

In [17]:
# print it more nicely

pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained.',
 'features': [{'center': [-122.25488, 37.87082],
               'context': [{'id': 'postcode.1038777018848930', 'text': '94720'},
                           {'id': 'place.4062647275990170',
                            'text': 'Berkeley',
                            'wikidata': 'Q484678'},
                           {'id': 'region.3591',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'wikidata': 'Q99'},
                           {'id': 'country.3145',
                            'short_code': 'us',
                            'text': 'United States',
                            'wikidata': 'Q30'}],
          

In [18]:
# pull out the lat-lon coordinates

for r in data['features']:
    coords = r['geometry']['coordinates']
    print(coords)

[-122.25488, 37.87082]


# Using Mapbox Python SDK for Geocoding

So far the discussion of APIs has been based on just accessing the API endpoints, which could be done with anything that can access a properly specified url.  Even just your browser.

Now we will look at APIs from a different perspective, one that is more Pythonic.  We will first need to install the Mapbox SDK.

In [3]:
#!pip install mapbox

    100% |████████████████████████████████| 3.7MB 4.6MB/s ta 0:00:011
    100% |████████████████████████████████| 143kB 11.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for iso3166 ... done
  Stored in directory: /Users/Paul/Library/Caches/pip/wheels/05/75/28/62c7e94498ff866a39dbaec5db14fc5b3214a6506e69254a63
  Running setup.py bdist_wheel for cachecontrol ... done
  Stored in directory: /Users/Paul/Library/Caches/pip/wheels/36/bd/5f/dbbee4f2d51f97ecd12a363f870361179cb1fd4bc1174ea08a
  Running setup.py bdist_wheel for msgpack ... done
  Stored in directory: /Users/Paul/Library/Caches/pip/wheels/f0/02/4c/525b56fce78c415eb8066f6554f9de02792df26b8f882f6d65
Successfully built iso3166 cachecontrol msgpack
  Found existing installation: botocore 1.5.92
    Uninstalling botocore-1.5.92:
      Successfully uninstalled botocore-1.5.92
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


You will need to manage access tokens for Mapbox APIs (and for many others).  Read this for more information:

https://github.com/mapbox/mapbox-sdk-py/blob/master/docs/access_tokens.md

### Forward Geocoding

Forward geocoding is the one we have looked at so far using an ALI endpoint.  Let's look at it again using the Mapbox Python SDK.

In [14]:
from mapbox import Geocoder
import os

In [7]:
os.environ['MAPBOX_ACCESS_TOKEN'] = "pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w"

In [15]:
os.environ['MAPBOX_ACCESS_TOKEN']

'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'

In [16]:
geocoder = Geocoder(access_token = os.environ['MAPBOX_ACCESS_TOKEN'])

In [17]:
geocoder.session.params['access_token'] == os.environ['MAPBOX_ACCESS_TOKEN']

True

In [24]:
response = geocoder.forward('Wurster Hall, Berkeley, CA', limit=1)

In [25]:
results = response.text

In [26]:
data = json.loads(results)
pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained. POI(s) provided '
                'by Foursquare.',
 'features': [{'bbox': [-71.614667973279,
                        45.4559916754476,
                        -71.3081929310576,
                        45.6046983089612],
               'center': [-71.5308, 45.5352],
               'context': [{'id': 'region.3648',
                            'short_code': 'CA-QC',
                            'text': 'Quebec',
                            'wikidata': 'Q176'},
                           {'id': 'country.3179',
                            'short_code': 'ca',
                            'text': 'Canada',
                            'wikidata': 'Q16'}],
               'geometry': {'coordinates': [

In [30]:
response = geocoder.forward(
    "washington", bbox=[-78.338320,38.520792,-77.935454,38.864909])
first = response.geojson()['features'][0]
print(first['place_name'])
# 'Washington, Virginia, United States'
print([round(coord, 3) for coord in first['geometry']['coordinates']])
# [-78.16, 38.71]

Washington, Virginia, United States
[-78.159, 38.714]


### Reverse Geocoding

Reverse geocoding does what it sounds like. It takes coordinates and returns an address.

In [55]:
response = geocoder.reverse(lon=-78.159, lat=38.714)

In [56]:
features = sorted(response.geojson()['features'], key=lambda x: x['place_name'])

In [33]:
for f in features:
    print('{place_name}: {id}'.format(**f))

456 Gay Street, Washington, Virginia 22747, United States: address.1788840445144522
United States: country.3145
Virginia, United States: region.228368
Washington, Virginia 22747, United States: postcode.6313359897262800
Washington, Virginia, United States: place.6320270976246050


In [43]:
response = geocoder.reverse(lon=-73.989, lat=40.733, limit=1, types=['country'])

In [42]:
features = response.geojson()['features']

In [36]:
print('{place_name}: {id}'.format(**features[0]))

United States: country.3145


In [45]:
import mapbox
help(mapbox.Directions)

Help on class Directions in module mapbox.services.directions:

class Directions(mapbox.services.base.Service)
 |  Access to the Directions v5 API.
 |  
 |  Method resolution order:
 |      Directions
 |      mapbox.services.base.Service
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  directions(self, features, profile='mapbox/driving', alternatives=None, geometries=None, overview=None, steps=None, continue_straight=None, waypoint_snapping=None, annotations=None, language=None, **kwargs)
 |      Request directions for waypoints encoded as GeoJSON features.
 |      
 |      Parameters
 |      ----------
 |      features : iterable
 |          An collection of GeoJSON features
 |      profile : str
 |          Name of a Mapbox profile such as 'mapbox.driving'
 |      alternatives : bool
 |          Whether to try to return alternative routes, default: False
 |      geometries : string
 |          Type of geometry returned (geojson, polyline, polyline6)
 |      overview : 

In [47]:
from mapbox import Directions

In [48]:
service = Directions()

In [49]:
origin = {
        'type': 'Feature',
        'properties': {'name': 'Portland, OR'},
        'geometry': {
        'type': 'Point',
        'coordinates': [-122.7282, 45.5801]}}
destination = {
    'type': 'Feature',
    'properties': {'name': 'Bend, OR'},
    'geometry': {
    'type': 'Point',
    'coordinates': [-121.3153, 44.0582]}}

In [51]:
response = service.directions([origin, destination],'mapbox/driving')

In [52]:
driving_routes = response.geojson()

In [54]:
driving_routes

{'features': [{'geometry': {'coordinates': [(45.57994, -122.72832),
     (45.56995, -122.69555),
     (45.52473, -122.66403),
     (45.53161, -122.56801),
     (45.54728, -122.55336),
     (45.53943, -122.41837),
     (45.45508, -122.37623),
     (45.3759, -122.2218),
     (45.36648, -122.15453),
     (45.37977, -122.04793),
     (45.30527, -121.87119),
     (45.3123, -121.79323),
     (45.30066, -121.73499),
     (45.15902, -121.66228),
     (45.10711, -121.55876),
     (45.05603, -121.51364),
     (45.02217, -121.51322),
     (44.86717, -121.42421),
     (44.78583, -121.32094),
     (44.76025, -121.22736),
     (44.71869, -121.22773),
     (44.72038, -121.17616),
     (44.65345, -121.12858),
     (44.45997, -121.19954),
     (44.27523, -121.16913),
     (44.05817, -121.31533)],
    'type': 'LineString'},
   'properties': {'distance': 269397.3, 'duration': 11626},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}